In [1]:
import nltk

In [2]:
reviews_train = []
for line in open('aclImdb/movie_data/full_train.txt', 'r'):
    reviews_train.append(line.strip())

In [3]:
import re

In [4]:
DEL_THESE = re.compile("(\#)|(\!)|(\;)|(\:)|(\")|(\@)|(\[)|(\])|(\()|(\))|(\$)|(\%)|(\^)|(\*)|(\.)|(\,)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
STOPWORDS = set(nltk.corpus.stopwords.words('english'))

In [5]:
for i in range(len(reviews_train)):
    reviews_train[i] =reviews_train[i].lower()

In [6]:
reviews_train = [REPLACE_WITH_SPACE.sub(' ',i) for i in reviews_train]
reviews_train = [DEL_THESE.sub('',i) for i in reviews_train]
for i in range(len(reviews_train)):
    reviews_train[i] = ' '.join([t for t in reviews_train[i].split() if t not in STOPWORDS])
    

In [7]:
reviews_train[2]

"brilliant acting lesley ann warren best dramatic hobo lady ever seen love scenes clothes warehouse second none corn face classic good anything blazing saddles take lawyers also superb accused turncoat selling boss dishonest lawyer pepto bolt shrugs indifferently i'm lawyer says three funny words jeffrey tambor favorite later larry sanders show fantastic mad millionaire wants crush ghetto character malevolent usual hospital scene scene homeless invade demolition site time classics look legs scene two big diggers fighting one bleeds movie gets better time see quite often"

In [8]:
X_test = []
for line in open('aclImdb/movie_data/full_test.txt', 'r'):
    X_test.append(line.strip())

In [9]:
for i in range(len(X_test)):
    X_test[i] =X_test[i].lower()
X_test = [REPLACE_WITH_SPACE.sub(' ',i) for i in X_test]
X_test = [DEL_THESE.sub('',i) for i in X_test]
for i in range(len(X_test)):
    X_test[i] = ' '.join([t for t in X_test[i].split() if t not in STOPWORDS])
    

In [11]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ayush/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
stm = nltk.stem.PorterStemmer()
lem = nltk.stem.WordNetLemmatizer()

In [12]:
reviews_train = [' '.join(stm.stem(word.decode('utf-8')) for word in reviews.split()) for reviews in reviews_train]


AttributeError: 'str' object has no attribute 'spilt'

In [31]:
X_test[7]

u'felt film mani good qualiti cinematographi certainli differ expos stage aspect set stori origin charact actor certainli achiev felt play quit convincingli cours play definit uniqu cultur aspect may leav mani disappoint familiar chines orient cultur answer lot question regard parent child relationship stigma goe drug use found jia hongsheng stori interest note stori beij fashion music reek earli 90 even though made 2001 realli cheesi sometim beatl crap etc whatev top ten twenti televis check'

In [30]:
X_test = [' '.join(stm.stem(word.decode('utf-8')) for word in reviews.split()) for reviews in X_test]


In [32]:
reviews_train[2]

u"brilliant act lesley ann warren best dramat hobo ladi ever seen love scene cloth warehous second none corn face classic good anyth blaze saddl take lawyer also superb accus turncoat sell boss dishonest lawyer pepto bolt shrug indiffer i'm lawyer say three funni word jeffrey tambor favorit later larri sander show fantast mad millionair want crush ghetto charact malevol usual hospit scene scene homeless invad demolit site time classic look leg scene two big digger fight one bleed movi get better time see quit often"

In [34]:
reviews_train = [' '.join(lem.lemmatize(word) for word in reviews.split()) for reviews in reviews_train]


In [41]:
X_test = [' '.join(lem.lemmatize(word) for word in reviews.split()) for reviews in X_test]

In [42]:
reviews_train[2]

u"brilliant act lesley ann warren best dramat hobo ladi ever seen love scene cloth warehous second none corn face classic good anyth blaze saddl take lawyer also superb accus turncoat sell bos dishonest lawyer pepto bolt shrug indiffer i'm lawyer say three funni word jeffrey tambor favorit later larri sander show fantast mad millionair want crush ghetto charact malevol usual hospit scene scene homeless invad demolit site time classic look leg scene two big digger fight one bleed movi get better time see quit often"

In [43]:
import sklearn

In [44]:
vec = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range = (1,3),max_df=0.9,min_df=7)

In [45]:
vec.fit(reviews_train)

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.9, max_features=None, min_df=7,
        ngram_range=(1, 3), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [46]:
X_train = vec.transform(reviews_train)
X_tst = vec.transform(X_test)
Y_train = [1 if i < 12500 else 0 for i in range(25000)]
Y_test = [1 if i < 12500 else 0 for i in range(25000)]

In [47]:
X_train.shape ,X_tst.shape

((25000, 67075), (25000, 67075))

In [48]:
X_trn,X_val,y_trn,y_val = sklearn.model_selection.train_test_split(X_train,Y_train,test_size = 0.2)

In [49]:
from sklearn.metrics import accuracy_score
clf = []
sc = 0
for c in [0.1,1,10,100]:
    lr = sklearn.linear_model.LogisticRegression(n_jobs = -1,C = c)
    lr.fit(X_trn,y_trn)
    y_pred = lr.predict(X_val)
    if(sc < accuracy_score(y_val,y_pred)):
        sc = accuracy_score(y_val,y_pred)
        clf = lr
    print(" C = ",c," score : ", accuracy_score(y_val,y_pred))
    

/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/logistic.py:1296: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


(' C = ', 0.1, ' score : ', 0.8586)
(' C = ', 1, ' score : ', 0.8868)
(' C = ', 10, ' score : ', 0.8916)
(' C = ', 100, ' score : ', 0.8886)


In [50]:
clf

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False)

In [51]:
from sklearn.naive_bayes import BernoulliNB
lr = BernoulliNB()
lr.fit(X_trn,y_trn)
y_pred = lr.predict(X_val)
print(" score : ", accuracy_score(y_val,y_pred))

(' score : ', 0.872)


In [53]:
clf = sklearn.linear_model.LogisticRegression(n_jobs = -1,C = 10,penalty='l2')
clf.fit(X_trn,y_trn)
y_pred = clf.predict(X_tst)
y_pred2 = clf.predict(X_train)
score = accuracy_score(Y_test,y_pred)
sc = accuracy_score(Y_train,y_pred2)
score,sc

(0.88508, 0.97592)

Our model is overfitting but let's see the imp coeeficent

In [54]:
feature_to_coef = {
    word: coef for word, coef in zip(
        vec.get_feature_names(), clf.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:7]:
    print (best_positive)
    
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:7]:
    print (best_negative)

(u'excel', 11.061159395744397)
(u'great', 10.103243317112769)
(u'perfect', 8.806294370402405)
(u'enjoy', 8.623245052091042)
(u'beauti', 7.671467513723754)
(u'love', 7.368625607116348)
(u'favorit', 7.206875205252072)
(u'worst', -14.134663026971294)
(u'aw', -12.0100689472504)
(u'bore', -11.792286337739505)
(u'wast', -11.298356621963745)
(u'bad', -11.112658266393318)
(u'poorli', -9.232277950393069)
(u'disappoint', -8.925042276318553)
